This notebook is a follow-along from this Medium post: 
    https://medium.com/@msalmon00/web-scraping-job-postings-from-indeed-96bd588dcb4b
    
Some of the code doesn't work correctly at the moment and I've made minor modifications, but the bulk works well enough to get started

In [1]:
import requests
import bs4
from bs4 import BeautifulSoup

import pandas as pd
import time

In [3]:
URL = "https://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"
#conducting a request of the stated URL above:
page = requests.get(URL)
#specifying a desired format of “page” using the html parser - this allows python to read the various components of the page, rather than treating it as one long string.
soup = BeautifulSoup(page.text, "html.parser")
#printing soup in a more structured tree format that makes for easier reading
print(soup.prettify())

<!DOCTYPE html>
<html dir="ltr" lang="en">
 <head>
  <meta content="text/html;charset=utf-8" http-equiv="content-type"/>
  <script id="polyfill-script-bundle">
   /* Disable minification (remove `.min` from URL path) for more info */

(function(self, undefined) {function ArrayCreate(r){if(1/r==-Infinity&&(r=0),r>Math.pow(2,32)-1)throw new RangeError("Invalid array length");var n=[];return n.length=r,n}function Call(t,l){var n=arguments.length>2?arguments[2]:[];if(!1===IsCallable(t))throw new TypeError(Object.prototype.toString.call(t)+"is not a function.");return t.apply(l,n)}function Get(n,t){return n[t]}function HasOwnProperty(r,t){return Object.prototype.hasOwnProperty.call(r,t)}function HasProperty(n,r){return r in n}function IsArray(r){return"[object Array]"===Object.prototype.toString.call(r)}function IsCallable(n){return"function"==typeof n}function RequireObjectCoercible(e){if(null===e||e===undefined)throw TypeError();return e}function SameValueNonNumber(e,n){return e===n}funct

In [6]:
def extract_job_title_from_result(soup): 
    jobs = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            jobs.append(a["title"])
    return(jobs)
extract_job_title_from_result(soup)

['Data Scientist (Personalization)',
 'Data Scientist',
 'Data Scientist',
 'Senior Data Scientist',
 'Junior Data Scientist',
 'Medical Data Scientist',
 'Data Scientist',
 'Associate Data Scientist, Ads',
 'Junior Data Scientist',
 'Predictive Maintenance Data Scientist',
 'Research Scientist-Center for Data Science',
 'Data Scientist',
 'Credit Quantitative Analyst/Data Scientist',
 'Data Scientist',
 'Assessment Content Contributor : Data Science']

In [8]:
def extract_company_from_result(soup): 
    companies = []
    for div in soup.find_all(name='div', attrs={'class':'row'}):
        company = div.find_all(name='span', attrs={'class':'company'})
        if len(company) > 0:
            for b in company:
                companies.append(b.text.strip())
        else:
            sec_try = div.find_all(name='span', attrs={'class':'result-link-source'})
            for span in sec_try:
                companies.append(span.text.strip())
    return(companies)
 
extract_company_from_result(soup)

['The New York Times',
 'FanDuel',
 'Thermo Fisher Scientific',
 'MasterCard',
 'Decode_M',
 'K Health',
 'Two Sigma Investments, LLC.',
 'Spotify',
 'CFSB',
 'Corning',
 'New York University',
 'Hinge',
 'Citi',
 'Altice',
 'Codecademy']

In [10]:
def extract_location_from_result(soup): 
    locations = []
    spans = soup.findAll('span', attrs={'class': 'location'})
    for span in spans:
        locations.append(span.text)
    return(locations)
extract_location_from_result(soup)

['New York, NY',
 'New York, NY',
 'New York, NY 10001 (Chelsea area)',
 'New York, NY 10011 (Flatiron District area)',
 'New York, NY',
 'New York, NY 10001 (Garment District area)',
 'New York, NY',
 'New York, NY',
 'New York, NY',
 'Corning, NY 14831',
 'New York, NY 10012 (Greenwich Village area)',
 'New York, NY 10010 (Park Slope area)',
 'New York, NY',
 'Long Island City, NY',
 'New York, NY']

In [12]:
# this part doesn't seem to work correctly
def extract_salary_from_result(soup): 
    salaries = []
    for div in soup.find_all(name='div', attrs={'class':'row'}):
        try:
            salaries.append(div.find('nobr').text)
        except:
            try:
                div_two = div.find(name='div', attrs={'class':'sjcl'})
                div_three = div_two.find('div')
                salaries.append(div_three.text.strip())
            except:
                salaries.append('Nothing_found')
    return(salaries)
extract_salary_from_result(soup)

['The New York Times\n\n\n\n4.0',
 'FanDuel\n\n\n\n4.1',
 'Thermo Fisher Scientific\n\n\n\n3.5',
 'MasterCard\n\n\n\n4.1',
 'Decode_M',
 'K Health',
 'Two Sigma Investments, LLC.\n\n\n\n3.8',
 'Spotify\n\n\n\n4.3',
 'CFSB\n\n\n\n4.4',
 'Corning\n\n\n\n3.8',
 'New York University\n\n\n\n4.2',
 'Hinge\n\n\n\n4.8',
 'Citi\n\n\n\n3.9',
 'Altice\n\n\n\n3.4',
 'Codecademy\n\n\n\n4.2']

In [13]:
def extract_summary_from_result(soup): 
    summaries = []
    spans = soup.findAll('span', attrs={'class': 'summary'})
    for span in spans:
        summaries.append(span.text.strip())
    return(summaries)
extract_summary_from_result(soup)

[]

In [19]:
max_results_per_city = 100
city_set = ['Chicago', 'Houston', 'Miami']
columns = ['city', 'job_title', 'company_name', 'location', 'summary', 'salary']
sample_df = pd.DataFrame(columns = columns)

In [20]:
sample_df

,city,job_title,company_name,location,summary,salary


In [24]:
new_array = []
for city in city_set:
    for start in range(0, max_results_per_city, 10):
        page = requests.get('http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=' + str(city) + '&start=' + str(start))
        time.sleep(1)  #ensuring at least 1 second between page grabs
        soup = BeautifulSoup(page.text, 'lxml', from_encoding='utf-8')
        for div in soup.find_all(name='div', attrs={'class':'row'}): 
            #specifying row num for index of job posting in dataframe
            num = (len(sample_df) + 1) 
            #creating an empty list to hold the data for each posting
            job_post = [] 
            #append city name
            job_post.append(city) 
            #grabbing job title
            for a in div.find_all(name='a', attrs={'data-tn-element':'jobTitle'}):
                job_post.append(a['title']) 
            #grabbing company name
            company = div.find_all(name='span', attrs={'class':'company'}) 
            if len(company) > 0: 
                for b in company:
                    job_post.append(b.text.strip()) 
            else: 
                sec_try = div.find_all(name='span', attrs={'class':'result-link-source'})
                for span in sec_try:
                    job_post.append(span.text) 
            #grabbing location name
            c = div.findAll('span', attrs={'class': 'location'}) 
            for span in c: 
                job_post.append(span.text) 
            #grabbing summary text
            d = div.findAll('span', attrs={'class': 'summary'}) 
            for span in d:
                job_post.append(span.text.strip()) 
            #grabbing salary
            try:
                job_post.append(div.find('nobr').text) 
            except:
                try:
                    div_two = div.find(name='div', attrs={'class':'sjcl'}) 
                    div_three = div_two.find('div') 
                    job_post.append(div_three.text.strip())
                except:
                    job_post.append('Nothing_found') 
            #appending list of job post info to dataframe at index num
            new_array.append(job_post)


In [28]:
pd.DataFrame(new_array,columns=sample_df.columns[:-1])

,city,job_title,company_name,location,summary
0,Chicago,Senior Data Scientist - Enterprise Data Science,"Blue Cross Blue Shield of IL, MT, NM, OK & TX","Chicago, IL","Blue Cross Blue Shield of IL, MT, NM, OK & TX\..."
1,Chicago,Senior Data Scientist - BAM Ventures,Balyasny,"Chicago, IL 60602 (The Loop area)",Balyasny\n\n\n\n4.0
2,Chicago,Data Scientist/Machine Learning Engineer- Remo...,"Cardinal Intellectual Property, Inc","Evanston, IL","Cardinal Intellectual Property, Inc"
3,Chicago,Senior Data Scientist - Nationwide Opportunities,"Amazon Web Services, Inc.","Chicago, IL","Amazon Web Services, Inc.\n\n\n\n3.5"
4,Chicago,Data Scientist,Colsh Consultants,"Chicago, IL",Colsh Consultants
...,...,...,...,...,...
205,Miami,AWS Data Engineer,Accenture,"Miami, FL 33126 (Flagami area)",Accenture\n\n\n\n4.0
206,Miami,Data Science Consultant,Accenture,"Miami, FL 33126 (Flagami area)",Accenture\n\n\n\n4.0
207,Miami,Data Governance and Quality Analytics & Cognit...,Deloitte,"Miami, FL 33131 (Downtown area)",Deloitte\n\n\n\n4.0
208,Miami,Google Cloud Data Engineer,Accenture,"Miami, FL 33126 (Flagami area)",Accenture\n\n\n\n4.0
